In [2]:
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
project_path = r'/content/gdrive/My Drive/Colab Notebooks/embedding_project/char_model/'

In [0]:
train_file = project_path + 'train_char.npy'
train_label = project_path + 'train_char_label.npy'
test_file =  project_path + 'test_char.npy'
test_label =  project_path + 'test_char_label.npy'

In [0]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, CuDNNLSTM, SpatialDropout1D, Dropout, Activation, Bidirectional, Conv1D, GlobalMaxPool1D
from tensorflow.keras.callbacks import ModelCheckpoint


def get_model():
    _vocab_size = 319
    _embedding_size = 300
    _hidden_layer_size = 64

    _model = Sequential()
    _model.add(Embedding(_vocab_size, _embedding_size, input_length=168))
    _model.add(SpatialDropout1D(0.3))
    _model.add(Bidirectional(CuDNNLSTM(_hidden_layer_size, return_sequences=True)))
    _model.add(Conv1D (kernel_size = (4), filters = 150, activation='relu'))
    _model.add(GlobalMaxPool1D())
    _model.add(Dense(50, activation="relu"))
    _model.add(Dropout(0.3))
    _model.add(Dense(20, activation="sigmoid"))
    _model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    _model.summary()
    return _model


X_train = np.load(train_file)
print(X_train.shape)
Y_train = np.load(train_label)
print(Y_train.shape)
X_test = np.load(test_file)
print(X_test.shape)
Y_test = np.load(test_label)
print(Y_test.shape)

model_file_path = r'conv_char_model.weights.best.hdf5'
model = get_model()
print(model)
model.load_weights(model_file_path)

checkpoint = ModelCheckpoint(filepath=model_file_path, save_best_only=True, verbose=1, monitor='val_loss')
BATCH_SIZE = 4096
history = model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE,
                    epochs=60,
                    validation_data=(X_test, Y_test),
                    callbacks=[checkpoint])

(loss, accuracy) = model.evaluate(X_test, Y_test)

In [23]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, CuDNNLSTM, SpatialDropout1D, MaxPooling1D, Dropout, Activation, Bidirectional, Conv1D, GlobalMaxPool1D
from tensorflow.keras.callbacks import ModelCheckpoint


def get_model():
    _vocab_size = 319
    _embedding_size = 300
    _hidden_layer_size = 64

    _model = Sequential()
    _model.add(Embedding(_vocab_size, _embedding_size, input_length=168))
    
    _model.add(Conv1D (kernel_size = (4), filters = 150, activation='relu'))
    _model.add(Dropout(0.3))
    _model.add(MaxPooling1D(3))
    
    _model.add(Bidirectional(CuDNNLSTM(_hidden_layer_size, return_sequences=False)))
    
    _model.add(Dense(20, activation="sigmoid"))
    _model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    _model.summary()
    return _model


# X_train = np.load(train_file)
# print(X_train.shape)
# Y_train = np.load(train_label)
# print(Y_train.shape)
# X_test = np.load(test_file)
# print(X_test.shape)
# Y_test = np.load(test_label)
# print(Y_test.shape)

model_file_path = r'conv_char_model.weights.best.hdf5'
model = get_model()
print(model)
model.load_weights(model_file_path)

checkpoint = ModelCheckpoint(filepath=model_file_path, save_best_only=True, verbose=1, monitor='val_loss')
BATCH_SIZE = 4096
history = model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE,
                    epochs=10,
                    validation_data=(X_test, Y_test),
                    callbacks=[checkpoint])

(loss, accuracy) = model.evaluate(X_test, Y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 168, 300)          95700     
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 165, 150)          180150    
_________________________________________________________________
dropout_15 (Dropout)         (None, 165, 150)          0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 55, 150)           0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 128)               110592    
_________________________________________________________________
dense_23 (Dense)             (None, 20)                2580      
Total params: 389,022
Trainable params: 389,022
Non-trainable params: 0
_________________________________________________________________
Trai